In [1]:
import pandas as pd
from lightning import pytorch as pl
from pathlib import Path
from chemprop import data, featurizers, models, nn
from chemprop.nn import metrics
from chemprop.models import multi
import matplotlib.pyplot as plt
import numpy as np
import torch
from lightning import pytorch as pl
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [7]:
import os
chemprop_dir = Path.cwd().parent
input_path = os.path.join(os.getcwd(), 'chemprop/regression.csv')
smiles_columns = 'smiles'
target_columns = ['logSolubility']

In [8]:
df_input = pd.read_csv(input_path)
df_input

,smiles,logSolubility
0,OCC3OC(OCC2OC(OC(C#N)c1ccccc1)C(O)C(O)C2O)C(O)...,-0.770
1,Cc1occc1C(=O)Nc2ccccc2,-3.300
2,CC(C)=CCCC(C)=CC(=O),-2.060
3,c1ccc2c(c1)ccc3c2ccc4c5ccccc5ccc43,-7.870
4,c1ccsc1,-1.330
...,...,...
495,Nc1cc(nc(N)n1=O)N2CCCCC2,-1.989
496,Nc2cccc3nc1ccccc1cc23,-4.220
497,c1ccc2cc3c4cccc5cccc(c3cc2c1)c45,-8.490
498,OC(c1ccc(Cl)cc1)(c2ccc(Cl)cc2)C(Cl)(Cl)Cl,-5.666


In [9]:
smiss = df_input.loc[:, smiles_columns].values
ys = df_input.loc[:, target_columns].values

In [10]:
smiss[:5], ys[:5]

(array(['OCC3OC(OCC2OC(OC(C#N)c1ccccc1)C(O)C(O)C2O)C(O)C(O)C3O',
        'Cc1occc1C(=O)Nc2ccccc2', 'CC(C)=CCCC(C)=CC(=O)',
        'c1ccc2c(c1)ccc3c2ccc4c5ccccc5ccc43', 'c1ccsc1'], dtype=object),
 array([[-0.77],
        [-3.3 ],
        [-2.06],
        [-7.87],
        [-1.33]]))

In [1]:
all_data = [[data.MoleculeDatapoint.from_smi(smis[0], y) for smis, y in zip(smiss, ys)]]
all_data += [[data.MoleculeDatapoint.from_smi(smis[i]) for smis in smiss] for i in range(1, len(smiles_columns))]

NameError: name 'smiss' is not defined

In [2]:
component_to_split_by = 0 
mols = [d.mol for d in all_data[component_to_split_by]]
train_indices, val_indices, test_indices = data.make_split_indices(mols, "random", (0.8, 0.1, 0.1))
train_data, val_data, test_data = data.split_data_by_indices(
    all_data, train_indices, val_indices, test_indices
)

NameError: name 'all_data' is not defined

In [3]:
featurizer = featurizers.SimpleMoleculeMolGraphFeaturizer()

train_datasets = [data.MoleculeDataset(train_data[0][i], featurizer) for i in range(len(smiles_columns))]
val_datasets = [data.MoleculeDataset(val_data[0][i], featurizer) for i in range(len(smiles_columns))]
test_datasets = [data.MoleculeDataset(test_data[0][i], featurizer) for i in range(len(smiles_columns))]

NameError: name 'featurizers' is not defined

In [4]:
train_mcdset = data.MulticomponentDataset(train_datasets)
scaler = train_mcdset.normalize_targets()
val_mcdset = data.MulticomponentDataset(val_datasets)
val_mcdset.normalize_targets(scaler)
test_mcdset = data.MulticomponentDataset(test_datasets)

NameError: name 'data' is not defined

In [5]:
train_loader = data.build_dataloader(train_mcdset)
val_loader = data.build_dataloader(val_mcdset, shuffle=False)
test_loader = data.build_dataloader(test_mcdset, shuffle=False)

NameError: name 'data' is not defined

In [6]:
mcmp = nn.MulticomponentMessagePassing(
    blocks=[nn.BondMessagePassing() for _ in range(len(smiles_columns))],
    n_components=len(smiles_columns),
)

NameError: name 'nn' is not defined

In [7]:
agg = nn.MeanAggregation()

NameError: name 'nn' is not defined

In [8]:
output_transform = nn.UnscaleTransform.from_standard_scaler(scaler)

NameError: name 'nn' is not defined

In [9]:
ffn = nn.RegressionFFN(
    input_dim=mcmp.output_dim,
    output_transform=output_transform,
)

NameError: name 'nn' is not defined

In [10]:
metric_list = [metrics.RMSEMetric(), metrics.MAEMetric()] 

NameError: name 'metrics' is not defined

In [11]:
mcmpnn = multi.MulticomponentMPNN(
    mcmp,
    agg,
    ffn,
    metrics=metric_list,
)

mcmpnn

NameError: name 'multi' is not defined

In [12]:
trainer = pl.Trainer(
    logger=False,
    enable_checkpointing=True,
    enable_progress_bar=True,
    accelerator="auto",
    devices=1,
    max_epochs=20, # number of epochs to train for
)

NameError: name 'pl' is not defined

In [13]:
trainer.fit(mcmpnn, train_loader, val_loader)

NameError: name 'trainer' is not defined

In [14]:
results = trainer.test(mcmpnn, test_loader)

NameError: name 'trainer' is not defined